In [27]:
import pandas as pd
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view

orig_df = pd.read_excel("一目均衡特徴量.xlsx",sheet_name=0,index_col=0)
feature_df = pd.DataFrame(index=orig_df.index)

def baseline_direction(**kwargs):
    orig = kwargs['col1'].reset_index(drop=False)
    tem_pd = pd.DataFrame(sliding_window_view(kwargs['col1'], 26))
    orig['基準線'] = (tem_pd.max(axis=1) + tem_pd.min(axis=1)) / 2
    orig['基準線'] = orig['基準線'].shift(25)
    tem_pd = pd.DataFrame(sliding_window_view(orig['基準線'].shift(-85).shift(1), 2)).fillna(value=0)
    orig['基準線方向'] = tem_pd[1]-tem_pd[0]
    orig['基準線方向'] = orig['基準線方向'].shift(85)
    return np.where(pd.isnull(orig['基準線方向']), np.NaN, orig['基準線方向'])

def reference_conversion_position(**kwargs):
    orig = kwargs['col1'].reset_index(drop=False)
    tem_pd = pd.DataFrame(sliding_window_view(kwargs['col1'], 9))
    orig['転換線'] = (tem_pd.max(axis=1) + tem_pd.min(axis=1)) / 2
    orig['転換線'] = orig['転換線'].shift(8)
    tem_pd = pd.DataFrame(sliding_window_view(kwargs['col1'], 26))
    orig['基準線'] = (tem_pd.max(axis=1) + tem_pd.min(axis=1)) / 2
    orig['基準線'] = orig['基準線'].shift(25)
    orig['基準転換位置'] = np.where(orig['転換線'] - orig['基準線'] > 0, 1, -1)
    orig['基準転換位置'] = orig['基準転換位置'].shift(-85).shift(85)
    return np.where(pd.isnull(orig['基準転換位置']), np.NaN, orig['基準転換位置'])


def reference_conversion_intersection(**kwargs):
    orig = kwargs['col1'].reset_index(drop=False)
    tem_pd = pd.DataFrame(sliding_window_view(kwargs['col1'].shift(-85).shift(1), 2)).fillna(value=0)
    orig['基準転換交差'] = tem_pd[1]-tem_pd[0]
    orig['基準転換交差'] = orig['基準転換交差'].shift(85)
    return np.where(pd.isnull(orig['基準転換交差']), np.NaN, orig['基準転換交差'])

def cloud_position(**kwargs):
    orig = kwargs['col1'].reset_index(drop=False)
    tem_pd = pd.DataFrame(sliding_window_view(kwargs['col1'], 9))
    orig['転換線'] = (tem_pd.max(axis=1) + tem_pd.min(axis=1)) / 2
    orig['転換線'] = orig['転換線'].shift(8)
    tem_pd = pd.DataFrame(sliding_window_view(kwargs['col1'], 26))
    orig['基準線'] = (tem_pd.max(axis=1) + tem_pd.min(axis=1)) / 2
    orig['基準線'] = orig['基準線'].shift(25)
    orig['先行スパン1'] = (orig['基準線'] + orig['転換線']) / 2
    orig['先行スパン1'] = orig['先行スパン1'].shift(-85).shift(26).fillna(value=0).shift(85)
    tem_pd = pd.DataFrame(sliding_window_view(kwargs['col1'], 52))
    # orig['MAX'] = tem_pd.max(axis=1)
    # orig['MIN'] = tem_pd.min(axis=1)
    orig['先行スパン2'] = (tem_pd.max(axis=1) + tem_pd.min(axis=1)) / 2
    orig['先行スパン2'] = orig['先行スパン2'].shift(-9).shift(1).fillna(value=0).shift(85)

    orig.to_excel('./t3.xlsx')
col1= orig_df['JPY T100 Curncy']
# feature_df['基準転換位置'] = reference_conversion_position(col1=col1)
# feature_df['基準転換交差'] = reference_conversion_intersection(col1=feature_df['基準転換位置'])
feature_df['基準転換交差'] = cloud_position(col1=col1)
# feature_df.to_excel('./feature_df.xlsx')

